# Week 9 Seminar -- Foundation Models for Financial Time Series

**Duration:** 90 minutes

| Exercise | Topic | Time |
|----------|-------|------|
| 1 | Chronos zero-shot on SPY | 25 min |
| 2 | Fine-tune a small model on financial data | 25 min |
| 3 | Hybrid: FM embeddings + XGBoost | 20 min |
| 4 | Discussion: Will FMs replace trees? | 20 min |

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 5)

print('Imports ready.')

In [ ]:
# Shared data loading
try:
    import yfinance as yf
    spy = yf.download('SPY', start='2020-01-01', end='2024-12-31', progress=False)
    spy_close = spy['Close'].dropna()
    spy_returns = spy_close.pct_change().dropna()
    print(f'Loaded {len(spy_close)} days of SPY data.')
except Exception as e:
    print(f'yfinance unavailable ({e}). Generating synthetic data.')
    np.random.seed(42)
    dates = pd.bdate_range('2020-01-01', '2024-12-31')
    returns = np.random.randn(len(dates)) * 0.012 + 0.0003
    spy_close = pd.Series(np.exp(np.cumsum(returns)) * 320, index=dates, name='Close')
    spy_returns = spy_close.pct_change().dropna()
    print(f'Generated {len(spy_close)} synthetic trading days.')

---
## Exercise 1: Chronos Zero-Shot on SPY (25 min)

**Goal:** Install Chronos, run zero-shot forecasting on SPY, and evaluate the results.

**Setup:**
```bash
pip install chronos-forecasting torch
```

### Tasks
1. Load the last 252 trading days as context
2. Forecast the next 21 trading days (one month)
3. Compare against a naive "last value" baseline
4. Compute RMSE and directional accuracy
5. Plot the forecast with confidence intervals

In [ ]:
# Task 1-2: Run Chronos zero-shot
import torch

context_length = 252  # 1 year
forecast_horizon = 21  # 1 month

# Split: use last forecast_horizon days as test
context_data = spy_close.iloc[-(context_length + forecast_horizon):-forecast_horizon]
test_data = spy_close.iloc[-forecast_horizon:]

print(f'Context: {context_data.index[0].date()} to {context_data.index[-1].date()} ({len(context_data)} days)')
print(f'Test:    {test_data.index[0].date()} to {test_data.index[-1].date()} ({len(test_data)} days)')

try:
    from chronos import ChronosPipeline

    pipeline = ChronosPipeline.from_pretrained(
        'amazon/chronos-t5-tiny',
        device_map='cpu',
        torch_dtype=torch.float32,
    )

    context_tensor = torch.tensor(context_data.values, dtype=torch.float32).unsqueeze(0)
    samples = pipeline.predict(context_tensor, prediction_length=forecast_horizon, num_samples=50)

    # Extract median, 10th, 90th percentiles
    chronos_median = samples.median(dim=1).values.squeeze().numpy()
    chronos_p10 = samples.quantile(0.1, dim=1).squeeze().numpy()
    chronos_p90 = samples.quantile(0.9, dim=1).squeeze().numpy()
    chronos_available = True
    print('Chronos forecast ready.')

except ImportError:
    print('Chronos not installed. Using simulated forecast for demonstration.')
    np.random.seed(77)
    last_val = context_data.values[-1]
    drift = np.linspace(0, 3, forecast_horizon)
    noise = np.cumsum(np.random.randn(forecast_horizon) * 0.7)
    chronos_median = last_val + drift + noise
    chronos_p10 = chronos_median - 5
    chronos_p90 = chronos_median + 5
    chronos_available = False

In [ ]:
# Task 3: Naive baseline (last value repeated)
naive_forecast = np.full(forecast_horizon, context_data.values[-1])

# Task 4: Evaluate
from sklearn.metrics import mean_squared_error

actual = test_data.values

rmse_chronos = np.sqrt(mean_squared_error(actual, chronos_median))
rmse_naive = np.sqrt(mean_squared_error(actual, naive_forecast))

dir_actual = np.sign(np.diff(actual))
dir_chronos = np.sign(np.diff(chronos_median))
dir_naive = np.sign(np.diff(naive_forecast))  # always 0 for naive

dir_acc_chronos = np.mean(dir_actual == dir_chronos)
dir_acc_naive = 0.0  # naive predicts no change

print(f'{"Metric":<20} {"Chronos":>10} {"Naive":>10}')
print('-' * 42)
print(f'{"RMSE":<20} {rmse_chronos:>10.2f} {rmse_naive:>10.2f}')
print(f'{"Dir. Accuracy":<20} {dir_acc_chronos:>10.1%} {dir_acc_naive:>10.1%}')
print()
if rmse_chronos > rmse_naive:
    print('Chronos has HIGHER RMSE than naive -- typical for zero-shot on finance.')
else:
    print('Chronos beats naive on RMSE -- unusual, worth investigating further.')

In [ ]:
# Task 5: Plot with confidence intervals
fig, ax = plt.subplots(figsize=(12, 5))

# Context tail
tail_len = 60
ax.plot(context_data.index[-tail_len:], context_data.values[-tail_len:],
        color='black', linewidth=1.5, label='Context (historical)')

# Actual test
ax.plot(test_data.index, actual, color='black', linewidth=1.5, linestyle='-',
        marker='o', markersize=3, label='Actual')

# Chronos forecast
ax.plot(test_data.index, chronos_median, color='indianred', linestyle='--',
        marker='s', markersize=3, label='Chronos (median)')
ax.fill_between(test_data.index, chronos_p10, chronos_p90, alpha=0.2,
                color='indianred', label='Chronos (80% CI)')

# Naive
ax.axhline(y=naive_forecast[0], color='steelblue', linestyle=':', label='Naive (last value)')

ax.axvline(x=test_data.index[0], color='gray', linestyle='--', alpha=0.5)
ax.set_title('Exercise 1: Chronos Zero-Shot Forecast on SPY', fontweight='bold')
ax.set_ylabel('Price')
ax.legend(loc='best')
plt.tight_layout()
plt.show()

### Exercise 1 -- Discussion Points

- Does Chronos capture the trend? The volatility?
- How wide are the confidence intervals relative to actual moves?
- Would this be tradeable? What Sharpe could you achieve?

---

## Exercise 2: Fine-Tuning a Small Model on Financial Data (25 min)

**Goal:** Fine-tune the Chronos-tiny model on financial return data and see if it improves.

> **Note:** Full fine-tuning of TimesFM (200M params) requires a GPU. Here we work with Chronos-tiny (8M) which fits on a CPU/M4 MacBook. If compute is limited, we walk through the setup and discuss results.

### Approach
1. Prepare multiple stock return series as training data
2. Fine-tune Chronos-tiny on these series for a few epochs
3. Evaluate fine-tuned model vs. zero-shot on held-out data

In [ ]:
# Prepare financial training data: multiple tickers
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META', 'NVDA', 'JPM', 'BAC', 'XOM', 'JNJ']

try:
    import yfinance as yf
    price_data = yf.download(tickers, start='2018-01-01', end='2024-06-30', progress=False)['Close']
    price_data = price_data.dropna()
    print(f'Downloaded data for {len(tickers)} tickers, {len(price_data)} days each.')
except Exception:
    np.random.seed(42)
    dates = pd.bdate_range('2018-01-01', '2024-06-30')
    price_data = pd.DataFrame(
        {t: np.exp(np.cumsum(np.random.randn(len(dates)) * 0.015 + 0.0003)) * (50 + i * 20)
         for i, t in enumerate(tickers)},
        index=dates
    )
    print(f'Generated synthetic data for {len(tickers)} tickers.')

print(f'Shape: {price_data.shape}')
price_data.tail(3)

In [ ]:
# Fine-tuning setup for Chronos
# Note: This requires the chronos-forecasting package with training support.
# The training API uses GluonTS data format.

print('=== Chronos Fine-Tuning Setup ===')
print()
print('Step 1: Prepare data in GluonTS format')
print('Step 2: Configure training (learning rate, epochs, batch size)')
print('Step 3: Run fine-tuning loop')
print('Step 4: Evaluate on held-out test set')
print()

# Prepare data as list of series (GluonTS-compatible format)
train_series = []
test_series = []
test_horizon = 21

for ticker in tickers:
    series = price_data[ticker].values
    train_series.append(series[:-test_horizon])
    test_series.append(series[-test_horizon:])

print(f'Prepared {len(train_series)} training series.')
print(f'Each training series: ~{len(train_series[0])} time steps.')
print(f'Test horizon: {test_horizon} steps.')

In [ ]:
# Fine-tuning demonstration
# In practice, you would run:
#
#   from chronos.training import train
#   train(
#       model_id='amazon/chronos-t5-tiny',
#       training_data=train_dataset,
#       output_dir='./chronos-finetuned',
#       learning_rate=1e-4,
#       num_epochs=5,
#       per_device_train_batch_size=8,
#   )
#
# For this seminar, we simulate the fine-tuning improvement.

print('Fine-tuning code (for reference):')
print()
print('from chronos.training import train')
print()
print('train(')
print('    model_id="amazon/chronos-t5-tiny",')
print('    training_data=train_dataset,')
print('    output_dir="./chronos-finetuned",')
print('    learning_rate=1e-4,')
print('    num_epochs=5,')
print('    per_device_train_batch_size=8,')
print(')')
print()
print('Typical fine-tuning time on M4 MacBook: ~15-30 min for tiny model.')
print('Expected improvement: 10-30% RMSE reduction vs. zero-shot.')

In [ ]:
# Simulated comparison: zero-shot vs fine-tuned vs ARIMA
np.random.seed(42)

results = []
for i, ticker in enumerate(tickers):
    actual = test_series[i]
    # Simulated RMSE values (based on typical literature results)
    base_rmse = np.std(np.diff(actual)) * np.sqrt(test_horizon) * 0.5
    results.append({
        'Ticker': ticker,
        'ARIMA RMSE': base_rmse * (0.9 + np.random.rand() * 0.3),
        'Chronos ZS RMSE': base_rmse * (1.0 + np.random.rand() * 0.4),
        'Chronos FT RMSE': base_rmse * (0.85 + np.random.rand() * 0.25),
    })

results_df = pd.DataFrame(results).set_index('Ticker')
print('Simulated RMSE Comparison (illustrative):')
print(results_df.round(2).to_string())
print()
print('Average improvement from fine-tuning: '
      f'{(1 - results_df["Chronos FT RMSE"].mean() / results_df["Chronos ZS RMSE"].mean()):.1%}')

In [ ]:
# Visualize the comparison
fig, ax = plt.subplots(figsize=(12, 5))

x = np.arange(len(tickers))
width = 0.25

ax.bar(x - width, results_df['ARIMA RMSE'], width, label='ARIMA', color='steelblue')
ax.bar(x, results_df['Chronos ZS RMSE'], width, label='Chronos (zero-shot)', color='indianred')
ax.bar(x + width, results_df['Chronos FT RMSE'], width, label='Chronos (fine-tuned)', color='green')

ax.set_xticks(x)
ax.set_xticklabels(tickers)
ax.set_ylabel('RMSE')
ax.set_title('Exercise 2: Zero-Shot vs. Fine-Tuned Chronos (Simulated)', fontweight='bold')
ax.legend()
plt.tight_layout()
plt.show()

### Exercise 2 -- Key Takeaways

- Fine-tuning helps, but does not close the gap to finance-native models
- The tokenizer is still calibrated for generic distributions
- With limited financial data, overfitting is a real risk
- Consider: is the improvement worth the compute cost vs. just using ARIMA?

---

## Exercise 3: Hybrid Approach -- FM Embeddings + XGBoost (20 min)

**Goal:** Extract embeddings from a foundation model and use them as features for XGBoost.

**Key insight:** Even if the FM is bad at *forecasting* financial data, its internal representations might capture useful temporal patterns that XGBoost can exploit.

### Pipeline
```
Price window --> FM encoder --> embedding vector (256-dim)
                                      |
Price window --> hand-crafted features (momentum, vol, etc.)
                                      |
                                      v
                              Concatenate --> XGBoost --> return prediction
```

In [ ]:
# Step 1: Create hand-crafted features

def make_features(prices, lookback=20):
    """Create standard alpha features from price series."""
    df = pd.DataFrame({'close': prices})
    df['ret_1d'] = df['close'].pct_change(1)
    df['ret_5d'] = df['close'].pct_change(5)
    df['ret_20d'] = df['close'].pct_change(20)
    df['vol_20d'] = df['ret_1d'].rolling(20).std()
    df['vol_60d'] = df['ret_1d'].rolling(60).std()
    df['mom_12_1'] = df['close'].pct_change(252) - df['close'].pct_change(21)
    df['sma_ratio'] = df['close'] / df['close'].rolling(50).mean()
    df['high_20d'] = df['close'].rolling(20).max() / df['close'] - 1
    df['low_20d'] = df['close'] / df['close'].rolling(20).min() - 1
    return df.dropna()

features_df = make_features(spy_close)
print(f'Created {features_df.shape[1] - 1} hand-crafted features.')
print(f'Features: {[c for c in features_df.columns if c != "close"]}')
features_df.tail(3)

In [ ]:
# Step 2: Generate FM embeddings
# In practice: pass rolling windows through the FM encoder and extract hidden states.
# Here we simulate this with a learned representation.

def extract_fm_embeddings(prices, window=60, embedding_dim=32):
    """
    Simulate FM embedding extraction.
    
    In practice, you would:
      1. Take a rolling window of prices
      2. Pass through the FM encoder (e.g., Chronos encoder)
      3. Extract the last hidden state or pooled representation
    
    Real code (with Chronos):
        model = ChronosModel.from_pretrained('amazon/chronos-t5-tiny')
        for i in range(window, len(prices)):
            context = prices[i-window:i]
            with torch.no_grad():
                hidden = model.encoder(tokenize(context))
                embedding = hidden.last_hidden_state.mean(dim=1)  # pool
            embeddings.append(embedding)
    """
    np.random.seed(42)
    n = len(prices) - window
    
    # Simulate: embeddings that capture some temporal structure
    returns = np.diff(np.log(prices))
    embeddings = np.zeros((n, embedding_dim))
    
    for i in range(n):
        window_returns = returns[i:i + window]
        # First few dims: statistical summaries (what an FM might learn)
        embeddings[i, 0] = np.mean(window_returns)
        embeddings[i, 1] = np.std(window_returns)
        embeddings[i, 2] = np.mean(window_returns[-5:])
        embeddings[i, 3] = np.mean(window_returns[-20:])
        # Remaining dims: nonlinear features (what deep learning captures)
        for j in range(4, embedding_dim):
            embeddings[i, j] = np.tanh(
                np.dot(window_returns[::window // (j + 1)][:5],
                       np.random.randn(5) * 0.1)
            )
    
    idx = prices.index[window:] if hasattr(prices, 'index') else range(window, len(prices))
    cols = [f'emb_{i}' for i in range(embedding_dim)]
    return pd.DataFrame(embeddings, index=idx, columns=cols)

emb_df = extract_fm_embeddings(spy_close, window=60, embedding_dim=32)
print(f'Generated embeddings: {emb_df.shape}')
emb_df.head(3)

In [ ]:
# Step 3: Combine features and train XGBoost
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Align features and embeddings by date
feature_cols = [c for c in features_df.columns if c != 'close']
combined = features_df[feature_cols].join(emb_df, how='inner')

# Target: next-day return
combined['target'] = spy_close.pct_change().shift(-1)
combined = combined.dropna()

# Train/test split (time-based)
split_idx = int(len(combined) * 0.8)
X_train = combined.iloc[:split_idx].drop(columns='target')
y_train = combined.iloc[:split_idx]['target']
X_test = combined.iloc[split_idx:].drop(columns='target')
y_test = combined.iloc[split_idx:]['target']

print(f'Train: {len(X_train)} samples, Test: {len(X_test)} samples')
print(f'Total features: {X_train.shape[1]} ({len(feature_cols)} hand-crafted + {emb_df.shape[1]} embeddings)')

In [ ]:
# Train three models: hand-crafted only, embeddings only, combined

xgb_params = dict(
    n_estimators=200, max_depth=4, learning_rate=0.05,
    subsample=0.8, colsample_bytree=0.8, random_state=42
)

emb_cols = [c for c in emb_df.columns]

models = {
    'Hand-crafted only': (feature_cols, XGBRegressor(**xgb_params)),
    'FM embeddings only': (emb_cols, XGBRegressor(**xgb_params)),
    'Combined (hybrid)': (feature_cols + emb_cols, XGBRegressor(**xgb_params)),
}

results = []
predictions = {}

for name, (cols, model) in models.items():
    usable_cols = [c for c in cols if c in X_train.columns]
    model.fit(X_train[usable_cols], y_train)
    y_pred = model.predict(X_test[usable_cols])
    predictions[name] = y_pred
    
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    ic = np.corrcoef(y_test, y_pred)[0, 1]
    r2 = r2_score(y_test, y_pred)
    dir_acc = np.mean(np.sign(y_test) == np.sign(y_pred))
    
    results.append({'Model': name, 'RMSE': rmse, 'IC': ic, 'R2': r2, 'Dir Acc': dir_acc})
    print(f'{name}: IC={ic:.4f}, R2={r2:.4f}, Dir Acc={dir_acc:.1%}')

results_df = pd.DataFrame(results).set_index('Model')
print()
print(results_df.round(4).to_string())

In [ ]:
# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# IC comparison
colors = ['steelblue', 'indianred', 'green']
axes[0].bar(results_df.index, results_df['IC'], color=colors, edgecolor='white')
axes[0].set_ylabel('Information Coefficient')
axes[0].set_title('IC Comparison')
axes[0].tick_params(axis='x', rotation=15)

# Feature importance for hybrid model
hybrid_model = models['Combined (hybrid)'][1]
all_cols = feature_cols + emb_cols
usable = [c for c in all_cols if c in X_train.columns]
importances = hybrid_model.feature_importances_

# Aggregate: hand-crafted vs. embedding importance
hc_imp = sum(importances[i] for i, c in enumerate(usable) if c in feature_cols)
emb_imp = sum(importances[i] for i, c in enumerate(usable) if c in emb_cols)

axes[1].pie([hc_imp, emb_imp], labels=['Hand-crafted', 'FM Embeddings'],
            colors=['steelblue', 'indianred'], autopct='%1.0f%%', startangle=90)
axes[1].set_title('Feature Importance Split (Hybrid Model)')

plt.suptitle('Exercise 3: Hybrid FM Embeddings + XGBoost', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.show()

### Exercise 3 -- Key Takeaways

- FM embeddings add incremental value on top of hand-crafted features
- The improvement is typically modest (a few basis points of IC)
- The hybrid approach is more robust than using FMs alone for forecasting
- XGBoost effectively selects which embedding dimensions are useful

---

## Exercise 4: Discussion -- Will FMs Replace Trees for Quant Finance? (20 min)

### Discussion Questions

1. **Data efficiency:** Trees work well with 1,000-5,000 samples. FMs need millions for pre-training. Which is more practical for a single stock?

2. **Interpretability:** Regulators require model explainability (SR 11-7, EU AI Act). Can you explain what a 256-dim embedding captures? How does this compare to SHAP values on a tree?

3. **Signal decay:** Alpha signals in finance decay quickly (months, not years). If you fine-tune an FM, how fast does it go stale?

4. **Competitive dynamics:** If everyone uses the same open-source FM (Chronos, Kronos), does the alpha disappear? Is the data moat more important than the model?

5. **The hybrid question:** If the best approach is FM embeddings + XGBoost, does that mean trees are still doing the heavy lifting?

### Positions to consider

**Pro-FM camp:**
- FMs capture complex nonlinear patterns that hand-crafted features miss
- Transfer learning from vast data compensates for small financial datasets
- Kronos shows that finance-native FMs can dramatically outperform trees

**Pro-tree camp:**
- Trees + good features have been the workhorse of quant for a decade
- Explainability is non-negotiable for institutional investors
- The marginal improvement from FMs does not justify the complexity
- Data (alternative data, faster data) matters more than model architecture

**Pragmatist camp:**
- Use FMs as feature extractors, trees as decision-makers
- The pipeline matters more than any single component
- Invest in data infrastructure, not model architecture